In [8]:
import re
from tqdm import tqdm
from rnnmorph.predictor import RNNMorphPredictor
import nltk
from nltk.tokenize import word_tokenize
import pandas as pd

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Владислав\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
file = open('Capital_Karl_Marx.txt', 'r')
text = file.read()
text[0:10]

'Карл Генри'

In [3]:
words = word_tokenize(text)
words[0:10]

['Карл',
 'Генрих',
 'Маркс',
 'Капитал',
 'Книга',
 'первая',
 ':',
 'процесс',
 'производства',
 'капитала']

In [4]:
predictor = RNNMorphPredictor(language='ru')

In [5]:
len(words)

491957

<h3>Получения свойств слов с RNN</h3>

Эта часть работает долго, можно перейти к следующей, результаты сохранены заранее

In [22]:
res = predictor.predict(words)

1/1 [==============================] - 793s 793s/step


In [32]:
nf = []
word = []
pos = []
tag = []
score = []
for i in res:
    nf.append(i.normal_form)
    word.append(i.word)
    pos.append(i.pos)
    tag.append(i.tag)
    score.append(i.score)
df = pd.DataFrame({'normal_form': nf, 'word': word, 'pos': pos, 'tag':tag, 'score':score})
df.head()

,normal_form,word,pos,tag,score
0,карла,Карл,NOUN,Case=Nom|Gender=Masc|Number=Sing,0.987951
1,генрих,Генрих,NOUN,Case=Nom|Gender=Masc|Number=Sing,0.986695
2,маркс,Маркс,NOUN,Case=Nom|Gender=Masc|Number=Sing,0.832564
3,капитал,Капитал,NOUN,Case=Nom|Gender=Masc|Number=Sing,0.954661
4,книга,Книга,NOUN,Case=Nom|Gender=Fem|Number=Sing,0.999114


In [33]:
df.to_csv('RNNwords.csv', index=False)

<h3>Поиск пар слов по предоставленным условиям</h3>

In [6]:
df = pd.read_csv('RNNwords.csv')
df.head()

,normal_form,word,pos,tag,score
0,карла,Карл,NOUN,Case=Nom|Gender=Masc|Number=Sing,0.987951
1,генрих,Генрих,NOUN,Case=Nom|Gender=Masc|Number=Sing,0.986695
2,маркс,Маркс,NOUN,Case=Nom|Gender=Masc|Number=Sing,0.832564
3,капитал,Капитал,NOUN,Case=Nom|Gender=Masc|Number=Sing,0.954661
4,книга,Книга,NOUN,Case=Nom|Gender=Fem|Number=Sing,0.999114


In [7]:
def get_tags_from_str(tag_str):
    params = tag_str.split('|')
    tags = {}
    if len(params)>1:
        for i in params:
            tag = i.split('=')
            tags[tag[0]] = tag[1]
    return tags

In [9]:
result = []
left_word = { 'word': '', 'POS': '', 'gender': '', 'number': '', 'case': '', 'idx':-2 }
idxs = df.index.to_list()

for i in tqdm(range(0, df.shape[0])):
    tag = get_tags_from_str(df.iloc[i].tag)
    if df.iloc[i].pos in ['NOUN', 'ADJ'] and len(set(tag.keys()).intersection(set(['Gender', 'Case', 'Number'])))==3:
        if (i-left_word['idx']==1 and left_word['gender']==tag['Gender'] and
            left_word['number']==tag['Number'] and left_word['case']==tag['Case'] and
            left_word['POS']!=df.iloc[i].pos):
            result.append(left_word['word'] + ' ' + df.iloc[i].normal_form)
        left_word['idx'] = i
        left_word['word'] = df.iloc[i].normal_form
        left_word['POS'] = df.iloc[i].pos
        left_word['gender'] = tag['Gender']
        left_word['number'] = tag['Number']
        left_word['case'] = tag['Case']

100%|████████████████████████████████████████████████████████████████████████| 491957/491957 [01:45<00:00, 4680.11it/s]


In [12]:
len(result)

27403

In [13]:
# Drop duplicates
result1 = list(set(result))
len(result1)

7520

In [14]:
file = open('task1_results.txt', 'r')
task1_result = file.read().split('\n')
task1_result[0:30]

['настоящий бедствие',
 'небольшой объём',
 'особенный форма',
 'возвращение свой',
 'наследственный привилегия',
 'этот половина',
 'этот нивелировка',
 'настоящий цель',
 'недостаточный понимание',
 'высокий смертность',
 'свой тонус',
 'парламентский отчёт',
 'действительный отдых',
 'адекватный бытие',
 'денежный маска',
 'иной отрасль',
 'увеличение рабочий',
 'какой результат',
 'малый остановка',
 'непрерывный кругооборот',
 'функциональный форма',
 'английский канцлер',
 'свободный труд',
 'буржуазный право',
 'общественный обмен',
 'узловой пункт',
 'широкий принцип',
 'свой накопление',
 'самый капитал',
 'идеальный образ']

In [15]:
res1_task1_diff = list(set(result1).difference(set(task1_result)))
res1_task1_diff.sort()
res1_task1_diff[0:30]

["'тый процесс",
 '.рабочий период',
 '000с подразделение',
 '1-й издание',
 '1-й миллион',
 '1-й продажа',
 '10-летний возраст',
 '10-часовой продолжительность',
 '101/2-часовой труд',
 '12-летний возраст',
 '12-часовой труд',
 '13-летний возраст',
 '13–14-часовой труд',
 '15-й неделя',
 '15-летний возраст',
 '15—55-летний возраст',
 '16-летний возраст',
 '18-й неделя',
 '18-летний возраст',
 '20-летний возраст',
 '21-й неделя',
 '23-й год',
 '24-часовой процесс',
 '30-й день',
 '30-летний возраст',
 '30-часовой работа',
 '37-й неделя',
 '4-й неделя',
 '4-й раздел',
 '4-летний ребёнок']

In [16]:
len(res1_task1_diff)

3179

<h3>Свойства RNN</h3>

- больше правильно определённых тегов на основе контекста
- понимаются сокращения, слова по смыслу схожие с прилагательными
- числительные принимаются за прилагательные
- понимает зашумлённые слова. Например: ру-ка, часто из-за этого возникают ошибки

In [17]:
task1_res1_diff = list(set(task1_result).difference(set(result1)))
task1_res1_diff.sort()
task1_res1_diff[0:30]

['-переменный капитал',
 '18-часовой день',
 'авансирование постоянный',
 'акт этот',
 'английский рабочий',
 'большинство свой',
 'бравый мозес',
 'ваш данные',
 'ваш детство',
 'ваш дух',
 'ваш идея',
 'ваш сведение',
 'ваш случай',
 'ваш утверждение',
 'введение новый',
 'ведение весь',
 'величина каждый',
 'величина сам',
 'величина свой',
 'величина тот',
 'величина этот',
 'весь англия',
 'весь буржуазия',
 'весь величина',
 'весь вероятность',
 'весь вес',
 'весь время',
 'весь год',
 'весь государство',
 'весь движение']

In [18]:
len(task1_res1_diff)

1480

<h3>Свойства PyMorphy</h3>

- местоимения выдаются за прилагательные

In [19]:
intersection = list(set(result1).intersection(set(task1_result)))
intersection.sort()
intersection[0:30]

['.больший часть',
 '/необходимый труд',
 '10-й неделя',
 '12-й неделя',
 '12-часовой день',
 '13-й неделя',
 '14-й неделя',
 '15-часовой период',
 '16-й неделя',
 '19-й неделя',
 '2-й неделя',
 '20-летний модистка',
 '220-футовый скорость',
 '24-часовой система',
 '3-й неделя',
 '5-й неделя',
 '5-футовый ход',
 '6-й неделя',
 '7-й неделя',
 '8-фунтовой ткань',
 '9-й неделя',
 'cобственный смысл',
 'iii-й случай',
 'абсолютный величина',
 'абсолютный граница',
 'абсолютный итог',
 'абсолютный количество',
 'абсолютный масса',
 'абсолютный обогащение',
 'абсолютный правило']

In [20]:
len(intersection)

4341

<h3>Объединение</h3>

- При объединении результаты получаются чище